In [1]:
import numpy as np

In [2]:
def indiff_mixed_action(payoff_matrix, own_supp, opp_supp, out):
    # (number of own actions, number of opponent's actions)
    nums_actions = payoff_matrix.shape
    
    # Support size
    k = len(own_supp)
    
    # Matrix in the left hand side of the linear equation
    a = np.empty((k+1, k+1))
    a[:-1, :-1] = payoff_matrix[own_supp, :][:, opp_supp]
    a[-1, :-1] = 1
    a[:-1, -1] = -1
    a[-1, -1] = 0
    
    # Vector in the right hand side of the linear equation
    b = np.zeros(k+1)
    b[-1] = 1
    
    try:
        sol = np.linalg.solve(a, b)
    except np.linalg.LinAlgError:
        return False
    
    
    # Return False immediately if any of the "probabilities" is not positive
    if (sol[:-1] <= 0).any():
        return False
    
    own_supp_c = np.setdiff1d(np.arange(nums_actions[0]), own_supp)
    # Return False immediately if the solution mixed action is not optimal
    if (sol[-1] < payoff_matrix[own_supp_c, :][:, opp_supp] @ sol[:-1]).any():
        return False
    
    out.fill(0)
    out[opp_supp] = sol[:-1]
    return True
    

In [3]:
import itertools

In [4]:
def support_enumeration(A, B_T):
    """
    Given a nondegenerate m x n bimatrix game (A, B_T), return a list of
    all Nash equilibria computed by the support enumeration algorithm.
    
    Parameters
    ----------
    A : ndarray(float, ndim=2)
        Payoff matrix for player 1, of shape (m, n).
    
    B_T : ndarray(float, ndim=2)
        Payoff matrix for player 2, of shape (n, m).
        
    Returns
    -------
    NEs : list(tuple(ndarray(float, ndim=1)))
        List containing tuples of Nash equilibrium mixed actions.
    
    """
    m, n = A.shape
    NEs = []
    for k in range(1, min(m, n)+1):
        for I in itertools.combinations(range(m), k):
            for J in itertools.combinations(range(n), k):
                y = np.empty(n)
                if indiff_mixed_action(A, list(I), list(J), y):
                    x = np.empty(m)
                    if indiff_mixed_action(B_T, list(J), list(I), x):
                        NEs.append((x, y))
    return NEs

In [5]:
A = np.array(
    [[    9504,     -660,    19976,   -20526,     1776,    -8976],
     [ -111771,    31680,  -130944,   168124,    -8514,    52764],
     [  397584,  -113850,   451176,  -586476,    29216,  -178761],
     [  171204,   -45936,   208626,  -263076,    14124,   -84436],
     [ 1303104,  -453420,  1227336, -1718376,    72336,  -461736],
     [  737154,  -227040,   774576, -1039236,    48081,  -300036]]
)
B_T = np.array(
    [[   72336,  -461736,  1227336, -1718376,  1303104,  -453420],
     [   48081,  -300036,   774576, -1039236,   737154,  -227040],
     [   29216,  -178761,   451176,  -586476,   397584,  -113850],
     [   14124,   -84436,   208626,  -263076,   171204,   -45936],
     [    1776,    -8976,    19976,   -20526,     9504,     -660],
     [   -8514,    52764,  -130944,   168124,  -111771,    31680]]
)

In [6]:
len(support_enumeration(A, B_T))

75